<a href="https://colab.research.google.com/github/chongzicbo/Dive-into-Deep-Learning-tf.keras/blob/master/4.2.%20%E6%A8%A1%E5%9E%8B%E5%8F%82%E6%95%B0%E7%9A%84%E8%AE%BF%E9%97%AE%E3%80%81%E5%88%9D%E5%A7%8B%E5%8C%96%E5%92%8C%E5%85%B1%E4%BA%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##4.2. 模型参数的访问、初始化和共享

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras import layers

In [0]:
tf.enable_eager_execution() #启用动态图计算

&emsp;&emsp;我们先定义一个与上一节中相同的含单隐藏层的多层感知机。我们依然使用默认方式初始化它的参数，并做一次前向计算。

In [0]:
net=Sequential([layers.Dense(256,activation='relu',input_shape=(20,)),layers.Dense(10)])
X=tf.random.uniform(shape=(2,20))
Y=net(X) #前向计算

In [4]:
Y.shape

TensorShape([Dimension(2), Dimension(10)])

###4.2.1. 访问模型参数
&emsp;&emsp;keras中访问模型参数有多种方法，首先我们可以通过net.trainable_weights属性获取模型的可训练参数。每个Dense层包括权重W和偏置bias,因此在trainable_weights中总共包括四个weight

In [5]:
for weight in net.trainable_weights:
  print(weight.shape)

(20, 256)
(256,)
(256, 10)
(10,)


&emsp;&emsp;我们也可以通过net.get_weights()方法来获取模型参数

In [6]:
for weight in net.get_weights():
  print(weight.shape)

(20, 256)
(256,)
(256, 10)
(10,)


&emsp;&emsp;此外，还可以先获取模型中的所有层，然后再获取各个层的参数。

In [7]:
for layer in net.layers:
  layer_name=layer.name
  print(layer_name)
  layer=net.get_layer(name=layer_name)
  print(len(layer.weights))
  print(layer.weights[0].shape)
  print(layer.weights[1].shape)

dense
2
(20, 256)
(256,)
dense_1
2
(256, 10)
(10,)


&emsp;&emsp;Dense中默认使用bias，且默认初始化为0。因为模型现在还没进行训练，所以bias全都为0。

In [8]:
net.get_layer('dense').get_weights()[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

###4.2.2 自定义模型参数初始化方法
&emsp;&emsp;模型参数初始化方法在3.15节已经讲过，这里主要介绍下自定义初始化方法。\
&emsp;&emsp;如果传递一个自定义的可调用函数，那么它必须使用参数 shape（需要初始化的变量的尺寸）和 dtype（数据类型）：

In [0]:
def my_init(shape,dtype=None):
  return tf.random.normal(shape=shape,dtype=dtype)

In [0]:
net=Sequential()
net.add(layers.Dense(64,kernel_initializer=my_init,input_shape=(10,)))

In [11]:
net.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 64)                704       
Total params: 704
Trainable params: 704
Non-trainable params: 0
_________________________________________________________________


###4.2.3. 共享模型参数
&emsp;&emsp;在有些情况下，我们希望在多个层之间共享模型参数。但是在前向传播函数call()中无法通过多次调用同一个层来计算，多次调用会报错。因此只能通过在call()函数中，将一个层的参数修改为同另一个层一样，以使两个层具有相同的模型参数。该功能通过set_weights()方法实现。

In [0]:
class MLPModel(keras.Model):
  def __init__(self):
    super(MLPModel,self).__init__()
    self.hidden=keras.layers.Dense(2,activation='relu') #隐藏层
    self.hidden1=keras.layers.Dense(2,activation='relu')   
    self.out=keras.layers.Dense(2) #输出层

  def call(self,inputs):
    x=self.hidden(inputs)
    x=self.hidden1(x)
    print(self.hidden.get_weights()) #将hidden1的参数修改为同hidden一样
    self.hidden1.set_weights(self.hidden.get_weights())
    x=self.out(x)
    return x #前向传播

In [0]:
x_in=tf.random.normal(shape=(2,2))
net=MLPModel()

In [14]:
net(x_in)

[array([[ 0.34257436, -0.21097231],
       [ 1.221043  ,  0.51139   ]], dtype=float32), array([0., 0.], dtype=float32)]


<tf.Tensor: id=216, shape=(2, 2), dtype=float32, numpy=
array([[0., 0.],
       [0., 0.]], dtype=float32)>

输出hidden和hidden1的参数，可以发现两个层的模型参数一样。

In [15]:
net.get_weights()

[array([[ 0.34257436, -0.21097231],
        [ 1.221043  ,  0.51139   ]], dtype=float32),
 array([0., 0.], dtype=float32),
 array([[ 0.34257436, -0.21097231],
        [ 1.221043  ,  0.51139   ]], dtype=float32),
 array([0., 0.], dtype=float32),
 array([[ 0.33180118, -0.6456049 ],
        [-0.7495526 ,  0.8344692 ]], dtype=float32),
 array([0., 0.], dtype=float32)]

4.2.4. 小结
* 有多种方法来访问模型参数。
* 可以自定义模型参数初始化方法。
* 模型参数可以共享。